<a href="https://colab.research.google.com/github/DharshiBalasubramaniyam/super-duper-rotary-phone/blob/main/googe-translate-and-gemini/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install libraries

In [ ]:
!pip install langchain langchain-community langchain-core pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.6/516.6 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install fastembed google-cloud-translate langchain-google-genai langchain-pinecone langchain-text-splitters python-dotenv

In [ ]:
import os
import time
import csv
import json
import re
import time

from pathlib import Path

from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from google.cloud import translate_v2 as translate

In [ ]:
def get_folder_path(folder_name):
    folder_path = os.path.join(os.getcwd(), folder_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    return folder_path


def detect_language(google_translate_client, text):
    language_response = google_translate_client.detect_language(text)
    return language_response["language"]


def translate_text(client, target_language, source_language, text):
    translate_response = client.translate(
        text,
        source_language=source_language,
        target_language=target_language,
    )
    return translate_response["translatedText"]

## Rag Evaluator

In [ ]:
class RagEvaluator:
    def __init__(self, llm):
        self.llm = llm

    def run(self, query, context, ground_truth, generated_response):
        prompt = f"""
        You are an expert evaluator tasked with assessing the quality of a generated response from a
        Cross-lingual Retrieval-Augmented Generation (RAG) system.

        The retrieved context is in English, while the query, generated response, and ground truth answer are
        all in the same language — either Tamil (ta), Sinhala (si), or English (en).

        The RAG system is explicitly instructed to respond with “I'm sorry, I can’t help with that.” if the retrieved context does not
        contain sufficient information to answer the query. In such cases, this response should be considered
        faithful and potentially correct depending on whether the answer was indeed absent in the context.

        For each of the following three dimensions in evaluation criteria, provide a score from 1 (very poor) to 5 (excellent),
        and explain your reasoning concisely in **English** only.

        ### Evaluation Criteria:

        - **Faithfulness**: Does the generated response accurately reflect the information in the retrieved context without introducing unsupported facts?
          → A response like “I'm sorry, I can’t help with that.” is highly faithful if the context truly lacks the necessary information.

        - **Answer Correctness**: How well does the generated response match the ground truth in terms of factual correctness and completeness?
          → If the ground truth cannot be derived from the context, and the generated response correctly says “I'm sorry, I can’t help with that.”, consider it correct.

        - **Context Relevance**: Is the retrieved context relevant and appropriate for answering the query?
          → If the context does not help in answering the query, score this low even if the response is appropriate.

        ### Output format (example)
        {{
          "faithfulness_score": "SCORE_BETWEEN_1_AND_5",
          "faithfulness_reason": "Explain whether the generated response stays true to the retrieved context or introduces hallucinations.",
          "answer_correctness_score": "SCORE_BETWEEN_1_AND_5",
          "answer_correctness_reason": "Explain how factually correct and complete the response is when compared with the ground truth.",
          "context_relevance_score": "SCORE_BETWEEN_1_AND_5",
          "context_relevance_reason": "Explain whether the retrieved context was appropriate and useful for answering the query."
        }}
        Make sure to return a JSON string like above without any formatting.

        ### Inputs:

        Below is the data you need to evaluate:

        Query: {query}

        Retrieved Context: {context}

        Generated Response: {generated_response}

        Ground Truth Answer: {ground_truth}
        """

        response = self.llm.invoke(prompt)

        return response


# llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key="")
# eval = RagEvaluator(llm)
# print(eval.run(
#     query="What are the requirements to get a driving license in Sri Lanka?",
#     generated_response="To obtain a driving license card in Sri Lanka, applicants must be 16 years or older and provide documents such as the birth certificate, Form K, and Grama Niladhari letter. The application is submitted to the Department for Registration of Persons.",
#     ground_truth="To get a driving license in Sri Lanka, you need to be 18 or older and submit Form K and your birth certificate to the Department for Registration of Persons.",
#     context="To obtain a driving license in Sri Lanka, you must be at least 18 years old, pass a written test and a practical driving test, and provide your NIC and medical certificate. The application must be submitted to the Department of Motor Traffic."
# ))


## Class RagChain

In [ ]:
class RagChain:
    def __init__(self, data_folder_name, pc_index_name, pinecone_api_key, google_api_key,
                 translate_client_config_file_path):
        self.data_folder_path = get_folder_path(data_folder_name)
        self.embedding_model = FastEmbedEmbeddings()
        self.vector_store = self.process_vector_store(pinecone_api_key, pc_index_name)
        self.llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)
        self.prompt_template = PromptTemplate(
            template="""You are a helpful and knowledgeable AI assistant that provides accurate and clear information about the registration of births, marriages, and deaths in Sri Lanka. You support responses in English (en), Sinhala (si), or Tamil (ta) based on the user's query language.

Use only the information from the retrieved documents to answer. Follow these instructions:

- Only answer questions related to registration information.
If the user's question is unrelated or the answer cannot be found in the retrieved documents, respond politely with:
"I'm sorry, I can’t help with that."

- Present the response in a clear, organized format using bullet points or numbered lists where appropriate.

- If relevant, include links from the 'Important Links' section (If exists) to guide users to official resources.

- At the end of the response, include a "For more information" link if available and relevant to the query.

Maintain a professional and helpful tone. Do not invent or assume information. Answer in the language used by the user in their query.
---
Context:
{context}

user: {question}
Assistant:
""",
            input_variables=["context", "question"],
        )
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            retriever=self.vector_store.as_retriever(search_kwargs={"k": 3}),
            chain_type="stuff",
            return_source_documents=True,
            chain_type_kwargs={"prompt": self.prompt_template},
        )
        self.google_translate_client = translate.Client.from_service_account_json(translate_client_config_file_path)

    def process_vector_store(self, pinecone_api_key, pc_index_name):
        pc = Pinecone(api_key=pinecone_api_key)

        existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

        if pc_index_name not in existing_indexes:
            print(f"Vector store index '{pc_index_name}' not found. Create new...")

            pc.create_index(
                name=pc_index_name,
                dimension=384,
                metric="cosine",
                spec=ServerlessSpec(
                    cloud="aws",
                    region="us-east-1"
                )
            )

            while not pc.describe_index(pc_index_name).status['ready']:  # Wait for the index to be ready
                time.sleep(1)
            print(f"Successfully created vector store index '{pc_index_name}'. Adding pdf files to new index...")

            index = pc.Index(pc_index_name)
            vector_store = PineconeVectorStore(index=index, embedding=self.embedding_model)

            print(f"Adding documents to index '{pc_index_name}'.")

            if os.path.exists(self.data_folder_path):
                md_files = list(Path(self.data_folder_path).rglob('*.md'))
                print(f"Found '{len(md_files)}' files to process.")
                for file_path in md_files:
                    print(f"Processing file: '{file_path}'.")
                    loader = TextLoader(file_path, encoding='utf-8')
                    documents = loader.load()

                    # splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
                    # chunks = splitter.split_documents(documents)
                    vector_store.add_documents(documents)
            else:
                print(f"Error: No found folder '{self.data_folder_path}'.")
            return vector_store

        print(f"Existing vector store found. Returning it: '{pc_index_name}'.")
        index = pc.Index(pc_index_name)
        return PineconeVectorStore(index=index, embedding=self.embedding_model)

    def process_query(self, query, user_input_language):
        english_query = query if user_input_language == "en" \
            else translate_text(self.google_translate_client, "en", user_input_language, query)
        return english_query

    def process_response(self, response, user_input_language):
        english_query = response if user_input_language == "en" \
            else translate_text(self.google_translate_client, user_input_language, "en", response)
        return english_query

    def run(self, user_input_query, ground_truth):
        user_input_language = detect_language(self.google_translate_client, user_input_query)
        query = self.process_query(user_input_query, user_input_language)
        llm_response = self.qa_chain.invoke({"query": f"{query}. Please provide your answer in {user_input_language}"})
        context = ""
        for doc in llm_response["source_documents"]:
            context += doc.page_content + "\n --- \n"
        eval = RagEvaluator(self.llm)
        eval_response = eval.run(
            query, context, ground_truth, llm_response['result']
        )

        return llm_response['result'], eval_response, context, query

## Initialize RAG chain

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
folder_name = "rgd_data"
pc_index_name = "rgd-data"
google_api_key = input("Enter google API kEY: ")
pinecone_api_key = input("Enter pinecone API key: ")
translate_client_config_file_path = input("Enter translate client config file path:")

Enter google API kEY: d
Enter pinecone API key: d
Enter translate client config file path:d


In [ ]:
rag_chain = RagChain(
    data_folder_name=folder_name,
    pc_index_name=pc_index_name,
    pinecone_api_key=pinecone_api_key,
    google_api_key=google_api_key,
    translate_client_config_file_path=translate_client_config_file_path
)


Existing vector store found. Returning it: 'rgd-data'.


## Evaluating the tamil QA pairs

In [ ]:
evaluation_data = [
    ['id', 'original_query', 'en_query', 'faithfulness_score', 'answer_correctness_score', 'context_relevance_score']
]

count = 1
qa_file_path = "/content/drive/My Drive/Colab Notebooks/fyrp/qa_data/tamil2.csv"
output_file_path = "/content/drive/My Drive/Colab Notebooks/fyrp/outputs/google_translate_ta.csv"
with open(qa_file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if count % 5 == 0:
            time.sleep(120)
        test_id = row['id']
        question = row['question']
        ground_truth = row['answer']

        print("\n----------------------\nprocessing question: ", question)

        generated_res, evaluation, context, en_query = rag_chain.run(question, ground_truth)
        print(f"""
Generate response:
{generated_res}

Evaluation:
{evaluation}

Context:
{context}
""")
        evaluation_match = re.search(r'\{.*\}', evaluation.strip(), re.DOTALL)
        if evaluation_match:
            try:
                result = json.loads(evaluation_match.group())
                faithfulness_score = result['faithfulness_score']
                answer_correctness_score = result['answer_correctness_score']
                context_relevance_score = result['context_relevance_score']
                test_res = [test_id, question, en_query, faithfulness_score, answer_correctness_score, context_relevance_score]
                evaluation_data.append(test_res)
            except json.JSONDecodeError as e:
                print("Failed to parse JSON:", e)
        else:
            print("Failed to match")
        count += 1
    with open(output_file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(evaluation_data)


----------------------
processing question:  ஒரு குழந்தையை தத்தெடுக்க தேவைப்பட்டால் தத்தெடுக்கக்கூடிய குழந்தையின் அதிகபட்ச வயது என்ன?

Generate response:
குழந்தையைத் தத்தெடுக்க விரும்பினால், தத்தெடுக்கப்பட வேண்டிய குழந்தையின் அதிகபட்ச வயது என்ன?

*   தத்தெடுக்கப்படும் குழந்தையின் வயது 14 வயதுக்கு மிகாமல் இருக்க வேண்டும்.

For more information: https://www.rgd.gov.lk/web/index.php?option=com_content&view=article&id=20&Itemid=151&lang=en#adoption-of-a-child

Evaluation:
```json
{
  "faithfulness_score": "5",
  "faithfulness_reason": "The generated response accurately reflects the information provided in the retrieved context, specifically stating that the age of the adopted child should not be more than 14 years. There are no unsupported facts introduced.",
  "answer_correctness_score": "5",
  "answer_correctness_reason": "The generated response perfectly matches the ground truth answer in terms of factual correctness and completeness. It directly answers the question about the maximum 

## Evaluating the sinhala QA pairs

In [ ]:
evaluation_data = [
    ['id', 'original_query', 'en_query', 'faithfulness_score', 'answer_correctness_score', 'context_relevance_score']
]

count = 1
qa_file_path = "/content/drive/My Drive/Colab Notebooks/fyrp/qa_data/sinhala2.csv"
output_file_path = "/content/drive/My Drive/Colab Notebooks/fyrp/outputs/google_translate_si.csv"
with open(qa_file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if count % 5 == 0:
            time.sleep(120)
        test_id = row['id']
        question = row['question']
        ground_truth = row['answer']

        print("\n----------------------\nprocessing question: ", question)

        generated_res, evaluation, context, en_query = rag_chain.run(question, ground_truth)
        print(f"""
Generate response:
{generated_res}

Evaluation:
{evaluation}

Context:
{context}
""")
        evaluation_match = re.search(r'\{.*\}', evaluation.strip(), re.DOTALL)
        if evaluation_match:
            try:
                result = json.loads(evaluation_match.group())
                faithfulness_score = result['faithfulness_score']
                answer_correctness_score = result['answer_correctness_score']
                context_relevance_score = result['context_relevance_score']
                test_res = [test_id, question, en_query, faithfulness_score, answer_correctness_score, context_relevance_score]
                evaluation_data.append(test_res)
            except json.JSONDecodeError as e:
                print("Failed to parse JSON:", e)
        else:
            print("Failed to match")
        count += 1
    with open(output_file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(evaluation_data)


----------------------
processing question:  දරුවෙකු තාත්තා ගැනීම සඳහා උපරිම වයස කීයද?

Generate response:
දරුවෙකු හදා වඩා ගැනීමේදී දරුවාගේ උපරිම වයස අවුරුදු 14 නොඉක්මවිය යුතුය.

*   හදා වඩා ගන්නා දරුවාගේ වයස අවුරුදු 14 ට වැඩි නොවිය යුතුය

වැඩි විස්තර සඳහා: [https://www.rgd.gov.lk/web/index.php?option=com\_content&view=article&id=20&Itemid=151&lang=en#adoption-of-a-child](https://www.rgd.gov.lk/web/index.php?option=com_content&view=article&id=20&Itemid=151&lang=en#adoption-of-a-child)

Evaluation:
```json
{
  "faithfulness_score": "5",
  "faithfulness_reason": "The generated response accurately reflects the information provided in the retrieved context regarding the maximum age for a child to be adopted.",
  "answer_correctness_score": "5",
  "answer_correctness_reason": "The generated response is factually correct and complete when compared with the ground truth. It correctly states that the maximum age for a child to be adopted is 14 years.",
  "context_relevance_score": "5",
  "con